In [1]:
#drive dependencies
!pip install xlrd
!pip install openpyxl
!pip install gensim
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#authenticate

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 112kB 2.7MB/s 
    100% |████████████████████████████████| 1.9MB 8.6MB/s 
  Running setup.py bdist_wheel for openpyxl ... - \ | done
  Stored in directory: /root/.cache/pip/wheels/57/41/b9/3765af8bda4a8d4b6aaf4957d7214984c3332348713e85cf36
  Running setup.py bdist_wheel for et-xmlfile ... - done
  Stored in directory: /root/.cache/pip/wheels/2a/77/35/0da0965a057698121fc7d8c5a7a9955cdbfb3cc4e2423cad39
Successfully built openpyxl et-xmlfile
    100% |████████████████████████████████| 23.6MB 1.5MB/s 
    100% |████████████████████████████████| 1.4MB 15.1MB/s 
    100% |████████████████████████████████| 133kB 29.6MB/s 
    100% |████████████████████████████████| 4.8MB 6.0MB/s 
    100% |████████████████████████████████| 61kB 24.1MB/s 
    100% |████████████████████████████████| 552kB 21.1MB/s 
  Running setup.py bdist_wheel for smart-open ... - done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa306

In [0]:
# downloading data from drive
data_excel = '15LcY9lrv5W7cX2HuUp1lhFbS4oxVN_2p'
downloaded = drive.CreateFile({'id': data_excel})
downloaded.GetContentFile('cik_list.xlsx')

In [0]:
# data dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import re
import openpyxl
import nltk
import urllib
import urllib.request
import requests
import time
# from pattern.en import ngrams

In [97]:
# reading data from excel sheet using pandas
ecl = pd.read_excel('cik_list.xlsx')
print(ecl.FORM.unique())
quarter = ecl.loc[ecl['FORM'].isin(['10-Q','NT 10-Q','10-Q/A'])]
year = ecl.loc[ecl['FORM'].isin(['10-K','NT 10-K','10-K/A', '10-K405/A', '10-K405'])]
print(quarter.shape)
# quarter = quarter.groupby('FORM')
print(year.shape)
# quarter.head(56)

['10-K405' '10-Q' 'NT 10-Q' '10-K/A' '10-Q/A' 'NT 10-K' '10-K' '10-K405/A']
(110, 6)
(42, 6)


In [98]:
# making a url list
url_begin = 'https://www.sec.gov/Archives/'
url_list = []
for i in list(ecl.SECFNAME):
  url_list.append(url_begin + i)
len(url_list)

152

In [0]:
# beautiful soup function for text extraction from web pages:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'}
def extraction(url):
  search = url
  home_page = requests.get(search,headers=headers)
  home_content = home_page.content
  soup = bs(home_content, 'html.parser')
  file = soup.text
  return file

# HTML tag remover
def cleanhtml(raw_html):
  cleantext = re.sub('(?s)<[^>]*>', '', raw_html)
  return cleantext

In [0]:
# Text data extraction
file=[]

for i in url_list:
  text = extraction(i)
  clean = cleanhtml(text)
  file.append(clean)

In [0]:
# new dataframe with complete url and extracted text

data = pd.DataFrame(columns=['Data', 'URL', 'FORM'])
data.Data = file
data.URL = url_list
data.FORM = ecl.FORM

In [101]:
data.iloc[34]['URL']

'https://www.sec.gov/Archives/edgar/data/4447/0000950123-04-003250.txt'

In [0]:
# making a dataframe to extract required sectional data:

section_data = pd.DataFrame(columns = ['MDA', 'QAQD', 'RFA'])
MDA = []
QAQD = []
RFA = []

In [0]:
# section extraction from file for further text analysis of the report:

def extraction_10K(file):
  patternMDA = re.search(r"(?s)Item 7[^A].*?Item 7", file).group()
  if patternMDA != None:
    MDA = patternMDA
  else:
    MDA = None
  patternQAQD = re.search(r"(?s)Item 7A.*?Item 8", file).group()
  if patternQAQD != None:
    QAQD = patternQAQ
  else:
    QAQD = None
  patternRFA = re.search(r"(?s)Item 1A.*?Item 1B", file).group()
  if patternRFA != None:
    RFA = patternRFA
  else:
    RFA = None
  return MDA,QAQD,RFA

def extraction_10Q(file):
  patternMDA = re.findall(r"(?s)Item\s*2.*?Item\s*3", file)
  if patternMDA[1] != None:
    MDA = patternMDA[1]
  else:
    MDA = None
  patternQAQD = re.findall(r"(?s)Item\s*2.*?Item\s*3", file)
  if patternQAQD[1] != None:
    QAQD = patternQAQD[1]
  else:
    QAQD = None
  patternRFA = re.findall(r"(?s)Item\s*1\s*a.*?Item\s*2", file)
  if patternRFA[1] != None:
    RFA = patternRFA[1]
  else:
    RFA = None
  

In [0]:
def section_extraction(file):
  if file['FORM'] == '10-Q' or 'NT 10-Q' or '10-Q/A':
    extraction_10Q(file['Data'])
  else:
    extraction_10K(file['Data'])

In [95]:
for i in range(len(data)):
  section_extraction(data.iloc[i])

TypeError: ignored

In [0]:
section_extraction(data)

In [43]:
# search for form type section in the file to extract and 
# check the form type of the form:

# Type = re.compile("CONFORMED SUBMISSION TYPE:.*")
# z = Type.search(data.Data[150]).group()
# z

'CONFORMED SUBMISSION TYPE:\t10-K/A'

In [63]:
# check form type of the report for further extraction:
res = z.split('\t').pop();
res


'10-K/A'

In [64]:
# compare with available types:
if res == '10-Q':
  print('true')
else:
  print('false')

false


In [0]:
# clean HTML tags from files:
file = data.Data[149]
clean = re.sub('(?s)<[^>]*>', '', file)
# print(clean)

In [0]:
for i in data.data:
  MDA.append(extraction_10_K(i)[0])
  QAQD.append(extraction_10_K(i)[1])
  RFA.append(extraction_10_K(i)[2])

In [0]:
section_data.MDA = MDA
section_data.QAQD = QAQD
section_data.RFA = RFA

In [6]:
# coding=utf8
# the above tag defines encoding for this document and is for Python 2.x compatibility
regex = r"(?s)Item 1.*?Item 2"

test_str = ("Item 1a\n"
	"Risk Factors\n"
	"Item 2\n"
	"13\n"
	"Item 1a\n"
	"akgjahotawiufhsdfaseuifa\n"
	"Item 2\n"
	"akjdhgaskughaorgagasrga\n"
	"Item 6")

matches = re.findall(regex, test_str)

if matches:
    print ("Match was found at {start}-{end}: {match}".format(start = matches.start(), end = matches.end(), match = matches.group()))
    
    for groupNum in range(0, len(matches.groups())):
        groupNum = groupNum + 1
        
        print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = matches.start(groupNum), end = matches.end(groupNum), group = matches.group(groupNum)))

# Note: for Python 2.7 compatibility, use ur"" to prefix the regex and u"" to prefix the test string and substitution.


AttributeError: ignored

In [5]:
matches

['Item 1a\nRisk Factors\nItem 2', 'Item 1a\nakgjahotawiufhsdfaseuifa\nItem 2']